In [1]:
import torch
import torch.nn as nn
import sys
sys.path.append("..")
from model_utils.custom_attentions import DotAtten_Layer

In [2]:
test = DotAtten_Layer(embed_dim=3, window_size=3, key_dim=3, value_dim=3)

input=torch.rand(4, 20, 3)
output=torch.rand(4, 20, 3)

lengths = torch.tensor([20, 20, 20, 10])
for i, l in enumerate([20, 20, 20, 10]):
    output[i,l:,:] = 0
    
output_hat = []
output_weights = []
query = torch.rand(4,1,3)

for i in range(20):
    mask, stop_edge = test.attention_mask(lengths, 20, query_index=i)
    
    if stop_edge is not None:
        input = torch.narrow(input, dim=0, start=0, length=stop_edge)
        query = torch.narrow(query, dim=0, start=0, length=stop_edge)
        mask = torch.narrow(mask, dim=0, start=0, length=stop_edge)
        blank_out = torch.zeros(4-stop_edge, 1, 3).type_as(out)
        blank_wgts = torch.zeros(4-stop_edge, *out_wgts.shape[1:]).type_as(out_wgts)
    
    out, out_wgts = test(query, key=input, value=input, attn_mask=mask)
    
    if stop_edge is not None:
        out = torch.cat([out, blank_out], dim=0)
        out_wgts = torch.cat([out_wgts, blank_wgts], dim=0)
        
    output_hat.append(out)
    output_weights.append(out_wgts)
    query = out
    
output_hat = torch.cat(output_hat, dim=1) 
output_weights = torch.cat(output_weights, dim=1) 

output_hat = nn.utils.rnn.pack_padded_sequence(output_hat, lengths=lengths, batch_first=True)
output = nn.utils.rnn.pack_padded_sequence(output, lengths=lengths, batch_first=True)

loss = (output[0] - output_hat[0]).abs().mean()
loss.backward()

test.wk.weight.grad

tensor([[ 2.5331e-03, -1.4304e-03, -1.6214e-03],
        [ 3.6288e-05,  7.2617e-05, -1.3357e-05],
        [ 1.8650e-03, -1.0807e-03, -1.1935e-03]])

In [3]:
from model_utils.network import deepspeech_LocalDotAttenLayers

In [4]:
input=torch.rand(4, 1000, 3)
lengths = torch.tensor([1000, 5, 4, 3])

output = torch.rand(4, 1000, 4)
for i, l in enumerate([1000, 5, 4, 3]):
    output[i,l:,:] = 0

input = nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=True)
output = nn.utils.rnn.pack_padded_sequence(output, lengths, batch_first=True)

In [5]:
test_dp = deepspeech_LocalDotAttenLayers(embed_dim=2, window_size=3, init_strategy="random", input_dim=3, output_dim=4, device="cpu")

In [6]:
output_hat, _, output_weight = test_dp(input, lengths)

In [7]:
test_dp.zero_grad()
loss = (output[0] - output_hat[0]).abs().mean()
loss
loss.backward()
test_dp.atten.wk.weight.grad

tensor([[0.0016, 0.0012],
        [0.0042, 0.0030]])

In [8]:
loss.ba

AttributeError: 'Tensor' object has no attribute 'ba'

In [9]:
test_dp.input_proj.weight.grad

tensor([[nan, nan, nan],
        [nan, nan, nan]])